In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0042378902435302734
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 17.835318771208208
200000 24.44117734373936
300000 25.386231241069837
400000 25.357780922657685
500000 27.01374418780298
600000 23.543483393803484
700000 22.95795849254224
800000 25.46356408287945
900000 24.30297852768909
1000000 11.507839328881994
1100000 21.375885662721306
1200000 22.773522251196685
1300000 23.041993817037614
1400000 22.84094022107628
1500000 24.854695346194433
1600000 22.810566152380723
1700000 23.452377261384036
1800000 23.129155860088762
1900000 23.388723571932257
2000000 8.179872614325914
2100000 25.39459093558505
2200000 28.28979300504732
2300000 27.118603577146015
2400000 27.61394404578327
2500000 27.18721273876328
2600000 26.883051961377728
2700000 26.349062131149353
2800000 28.00031372340222
2900000 25.15140592238408
3000000 12

29400000 23.389120333502298
29500000 23.879436390710524
29600000 10.414113443820696
29700000 23.01212020141026
29800000 23.311851497661845
29900000 22.16930746184
30000000 22.093603801380734
30100000 21.485463394877762
30200000 20.87624693646607
30300000 9.963654103734873
30400000 19.8163305700272
30500000 21.01969485121603
30600000 22.83611646661304
30700000 22.59707955269115
30800000 23.306948606679022
30900000 25.154688319078982
31000000 18.449131941637713
31100000 16.355323116579846
31200000 23.855940934873043
31300000 22.80826686042922
31400000 21.72136478995188
31500000 16.633658074782993
31600000 14.727195085247853
31700000 22.912456812129946
31800000 21.86914711674654
31900000 22.560685641159104
32000000 21.349494607969365
32100000 19.769186314990797
32200000 18.98278362621041
32300000 10.264962749939706
32400000 20.488691062655793
32500000 19.95506495974702
32600000 21.482672998568166
32700000 21.23709021401091
32800000 23.253503802971697
32900000 22.77629928742512
33000000 11

59200000 17.954937337366037
59300000 19.015439852074937
59400000 18.8153865901478
59500000 18.980804598398567
59600000 8.149394628875433
59700000 18.446355449317767
59800000 18.574311868234116
59900000 17.727862661440362
60000000 18.76118777630114
60100000 10.580947084482943
60200000 19.490764033138593
60300000 19.877656097408064
60400000 20.396237373632626
60500000 21.150091079610508
60600000 22.058620773652283
60700000 21.907651945902906
60800000 20.499879531744124
60900000 20.33037497460411
61000000 20.624143232950875
61100000 8.347905419295664
61200000 19.37979893056764
61300000 18.875229347565185
61400000 18.853010846683528
61500000 16.917728186409942
61600000 9.670790834647503
61700000 18.97795179120473
61800000 18.92934319752893
61900000 18.998033072752463
62000000 14.418854047338636
62100000 13.312939378696715
62200000 19.421086888880204
62300000 20.77389188149934
62400000 19.87835594658192
62500000 20.934306281209903
62600000 19.311715929936828
62700000 18.78952917826233
62800

89000000 16.587226876106755
89100000 16.28779659789706
89200000 9.083524690589176
89300000 16.371027771475955
89400000 15.96202071567446
89500000 16.034288588112172
89600000 17.473226680946993
89700000 17.829893732298046
89800000 17.811460208264933
89900000 17.8091183937055
90000000 18.494844878442994
90100000 17.329096825452307
90200000 16.02270432643414
90300000 16.78700238307972
90400000 15.960487168981805
90500000 15.529306680417697
90600000 7.879848506302841
90700000 15.490345092648317
90800000 13.488518078329367
90900000 8.61291628661638
91000000 14.816584752985209
91100000 8.874322284191821
91200000 15.337118765669048
91300000 16.53603677025221
91400000 15.065393176417071
91500000 15.803915267900397
91600000 16.852458438801776
91700000 16.891309746788448
91800000 16.62848997672515
91900000 17.192581238333155
92000000 17.359772000680906
92100000 16.295961727751997
92200000 15.411114327949488
92300000 15.75247387601317
92400000 15.277111365697486
92500000 16.232530565565742
926000

118000000 1.5793922868129664
118100000 1.6646586586440364
118200000 1.5417241870302558
118300000 1.5358789966197968
118400000 1.6503683775886973
118500000 1.8900820155913254
118600000 2.0234570139145087
118700000 1.7337404605350362
118800000 1.9112000750052134
118900000 1.5374510962866028
119000000 1.5307116471860083
119100000 1.619944149872911
119200000 1.5172658774530583
119300000 1.6047476092797703
119400000 1.7218689740266004
119500000 1.5090469968776603
119600000 1.5428735236629203
119700000 1.56076470405762
119800000 1.699733780895641
119900000 1.7086286024491868
120000000 1.5098076774514495
120100000 1.5305158471558318
120200000 1.5810551118264038
120300000 1.5244382066391549
120400000 1.5251468713329528
120500000 1.6092228192869804
120600000 2.1218328394535604
120700000 1.8214306673586373
120800000 1.6338869248134282
120900000 1.6711814550889135
121000000 1.6545074551100194
121100000 1.6337930072054432
121200000 1.5990775987335661
121300000 1.5599730526488647
121400000 1.548983

...

204500000 8.991381920027255
204600000 9.256700366770035
204700000 8.668428990543383
204800000 3.978274620280046
204900000 9.390752106335807
205000000 8.846517063343104
205100000 9.373385598425687
205200000 9.186148590132879
205300000 9.186789733892125
205400000 5.674194960074889
205500000 7.801649641685122
205600000 9.164432156576867
205700000 8.920958572737733
205800000 8.697493077903443
205900000 7.8442913373105725
206000000 7.932683100106196
206100000 2.954376734284077
206200000 8.129234144224462
206300000 7.896731178549104
206400000 7.957571388770199
206500000 8.259788261753899
206600000 8.420724402186726
206700000 8.118443519083197
206800000 4.727198486249727
206900000 8.577315225172294
207000000 8.685525203052528
207100000 8.482503948137945
207200000 8.585458045943888
207300000 7.563230783697111
207400000 5.347736595646538
207500000 8.768701394608208
207600000 8.207808995481638
207700000 7.913587168572421
207800000 7.646072184112603
207900000 7.304973405542992
208000000 5.7022359

233800000 6.398217066853402
233900000 6.463642072817973
234000000 6.847501609549242
234100000 6.438533578995406
234200000 6.547635120195683
234300000 4.530650203343523
234400000 5.367232763822405
234500000 3.340990285131958
234600000 6.608687775297398
234700000 6.552329004729553
234800000 6.467701065460754
234900000 6.539498464257405
235000000 6.302390172892006
235100000 6.220332611005581
235200000 6.1908928678353075
235300000 6.515740211695129
235400000 2.2344436410595496
235500000 6.6885557118396815
235600000 6.241501919304942
235700000 6.24430224211275
235800000 6.45929816442398
235900000 6.6534309093697965
236000000 6.508446257973124
236100000 6.576922427720171
236200000 6.499536146141895
236300000 2.7477468064120005
236400000 4.220361277885623
236500000 5.576887697270693
236600000 6.254908786409179
236700000 6.2904330505167545
236800000 6.492529215769827
236900000 6.452723629937162
237000000 6.114419702455533
237100000 6.011617722855564
237200000 6.195433927322144
237300000 4.6005

262400000 1.4576126079062817
262500000 1.8517997481920603
262600000 1.1557287087639265
262700000 0.499383351853417
262800000 0.5028360601835311
262900000 1.0409245841932715
263000000 1.9619961360346623
263100000 1.4620766801378131
263200000 1.6397168684311891
263300000 1.6715973131887416
263400000 1.5184809517938973
263500000 1.5797840988810552
263600000 1.570863320774907
263700000 1.8554500722162466
263800000 1.6904028607302402
263900000 1.6491065307198294
264000000 1.5480834901118516
264100000 1.5870109743496987
264200000 1.4896182216496394
264300000 1.5290733236908356
264400000 1.6373947111243352
264500000 1.4287941436883165
264600000 0.7875551487109482
264700000 0.4864835035699315
264800000 0.6386629755463429
264900000 1.3477325417654158
265000000 1.7907050261844628
265100000 1.549588711969326
265200000 1.4445946068118056
265300000 1.5327644005443268
265400000 1.5360311400825275
265500000 1.5226014934565346
265600000 1.5584043192112915
265700000 1.8324380868342254
265800000 1.55167

290900000 0.8437032166726178
291000000 0.933316674025309
291100000 0.9517974007040382
291200000 0.9298569798101312
291300000 0.8079648019073566
291400000 0.8225521860343125
291500000 0.841543891598932
291600000 0.7852243233206253
291700000 0.7262679093617995
291800000 0.9268055523399943
291900000 0.5575857715689778
292000000 0.24630124080412985
292100000 0.244840820893089
292200000 0.5114287342866659
292300000 0.92852190098127
292400000 0.706063839034468
292500000 0.8097537449987133
292600000 0.8612483299310625
292700000 0.7576541912731978
292800000 0.8189166987629951
292900000 0.8178631686644852
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 0.9154190271099494
293100000 0.8282275890102764
293200000 0.7668647722297258
293300000 0.7809531806280573
293400000 0.7768351358871798
293500000 0.6560927313870695
293600000 0.5883140983322157
293700000 0.6239095049402476
293800000 0.7221503185268051
293900000 0.5765435168051037
294000000 0.5575301298481922
2941000

317200000 0.023949387171005542
best so far: 0
type: [1, 1, 125, 1, 1] 125
cases of this type: 15625
best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.0303985394771768
317400000 0.065356408232824
317500000 0.05469201666453944
317600000 0.05545146232836114
317700000 0.05041625926325519
317800000 0.04788604739610619
317900000 0.04525068189490901
318000000 0.05153705203012228
318100000 0.03599847374936475
318200000 0.01642620722827779
318300000 0.015586999465320507
318400000 0.014823914773618512
318500000 0.02582589811901252
318600000 0.0354820940947632
318700000 0.03296238434124059
318800000 0.029886974898899927
318900000 0.024592932941687108
319000000 0.023595317678901225
319100000 0.01935249493845635
319200000 0.022139899467635154
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this type: 390625
319300000 0.014615598042825195
319400000 0.00815171477222244
319500000 0.007230446735638381
319600000 0.006610772464404504
best so far: 0
type: [1, 5, 1, 25, 1]